## Tải thư viện

In [ ]:
# Cài đặt gói Python từ SSI DC DATA. Địa chỉ Folder cài đặt thay đổi theo thực tế.
import subprocess
file_caidat = "C:\\Users\\DELL\\Desktop\\FCDATA\\dist\\ssi_fc_data-2.2.2.tar.gz"
subprocess.run(["pip", "install", file_caidat], check=True)

CompletedProcess(args=['pip', 'install', 'C:\\Users\\DELL\\Desktop\\FCDATA\\dist\\ssi_fc_data-2.2.2.tar.gz'], returncode=0)

In [9]:
# Import thư viện pandas và time
import pandas as pd
import time
from ssi_fc_data import fc_md_client , model

## Khởi tạo API

In [ ]:
# Lưu các key khởi tạo từ SSI API vào các biến
ConsumerID = 
ConsumerSecret = 
PublicKey = 
PrivateKey = 

In [19]:
# Import dataclass để tạo class lưu trữ dữ liệu một cách gọn gàng
from dataclasses import dataclass

# Định nghĩa một class lưu trữ dữ liệu cấu hình một cách tiện lợi
@dataclass
class Config:
    consumerID: str
    consumerSecret: str
    url: str

In [25]:
# Tạo một đối tượng cấu hình (Config) để xác thực và kết nối với API.
config = Config(
    consumerID= ConsumerID,
    consumerSecret=ConsumerSecret,
    url="https://fc-data.ssi.com.vn/"
)

# Khởi tạo MarketDataClient với config
client = fc_md_client.MarketDataClient(config)

In [8]:
# Kiểm client được khởi tạo.
client

In [27]:
# Lấy Access Token từ API
def md_access_token():
	print(client.access_token(model.accessToken(config.consumerID, config.consumerSecret)))

In [29]:
# Xuất kết quả access token
md_access_token()

{'message': 'Success', 'status': 200, 'data': {'accessToken': 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.ewogICJhdWQiOiAiZDMyNmVlMTgwNThiNDU1ODk4ZjIyZGQzYjA1OWZkZTYiLAogICJpYXQiOiAxNzM5OTc2OTkzLAogICJleHAiOiAxNzQwMDA1NzkzLAogICJzY29wZSI6IHsKICAgICJJcCI6IHsKICAgICAgIklwQWRyZXNzIjogIiIsCiAgICAgICJJc1ZhbGlkIjogdHJ1ZQogICAgfSwKICAgICJUYXJnZXQiOiBbCiAgICAgIHsKICAgICAgICAiVGFyZ2V0SWQiOiAiNWQxYjg5Y2JkNzdhNDVlNDkzOTI5YWExNmVlYzk3YzUiLAogICAgICAgICJUYXJnZXRVcmwiOiAiaHR0cHM6Ly9mYy1kYXRhLnNzaS5jb20udm4iLAogICAgICAgICJDb25uZWN0aW9uTmFtZSI6ICJGQ0RhdGEiCiAgICAgIH0KICAgIF0KICB9LAogICJjbGkiOiAiVXNlckFwcDE5OTkyNCIsCiAgInN1YiI6ICIxOTk5MjQiCn0.BfE6F_ns9R8uCFK0_BOcP9msPu-D5OW2ywzi7dkjStGFZP1dUln5at3KYKa6ZktoeUsiUadknh9W15aL9Ye_-maOF1um_GahO5bWdM4uUxKiLT0FOUiscLSrVlbKlivFwbmFzsxr_LhLXXkjZHdi1zKUrw3ChwDlLSBXXTyN3og0Mwh874-5YYDsVXMmNhtgEX4DuH3A9wvfNqs2LdyUXyFY6kHp_nvmUUpOSlq3y0DN2LhSmsa_ahi4_Gm14OBaumWAQRRxuypII3868oCUvTeIKpt0Cb9JqhN2_eSDRMc6DdRU8FsbIH765Uk-H4uIfgzJ3sQxi61Pt_5e8OGJbQ'}}


In [ ]:
# Lưu biến access token
accessToken = 

## Tạo dataframe chứa thông dữ liệu chứng khoán

In [79]:
# Khởi tạo danh sách mã chứng khoán thuộc chỉ số VN30 và xác định khoảng thời gian cần lấy dữ liệu.
# Danh sách mã chứng khoán và thời gian có thể tùy chỉnh nhu cầu.
VN30 = ["ACB", "BCM", "BID", "BVH", "CTG", "FPT", "GAS", "GVR", "HDB", "HPG",
    "KDH", "MBB", "MSN", "MWG", "NVL", "PDR", "PLX", "SAB", "SHB", "SSB",
    "SSI", "STB", "TCB", "TPB", "VCB", "VHM", "VIB", "VIC", "VJC", "VNM"
]
start_date = '01/01/2024'
end_date = '31/12/2024'

In [81]:
# Lấy dữ liệu giá chứng khoán hàng ngày (OHLC - Open, High, Low, Close) từ API
def md_get_daily_OHLC(stock, start_date, end_date):
	return client.daily_ohlc(config, model.daily_ohlc(stock, start_date, end_date, 1, 360, True))

In [141]:
# Tự động lấy dữ liệu giá hàng ngày (OHLC) cho toàn bộ VN30 và lưu vào dictionary.
vn30_dict = {}
for stock in VN30:
    # Lấy dữ liệu OHLC hàng ngày cho mỗi cổ phiếu
    result = md_get_daily_OHLC(stock, start_date, end_date)
    data = pd.DataFrame(result.get('data', []))  # Sử dụng .get() để tránh KeyError
    vn30_dict[f'{stock}'] = data
    time.sleep(1)

In [145]:
# Tạo một DataFrame tổng hợp dữ liệu OHLC của toàn bộ cổ phiếu trong VN30, sau đó nối dữ liệu từ từng cổ phiếu vào DataFrame chung (vn30_df).
columns = ['Symbol', 'TradingDate', 'Open', 'High', 'Low', 'Close', 'Volume', 'Value']
vn30_df = pd.DataFrame(columns=columns)
vn30_df
for stock in VN30:
    if stock in vn30_dict:
        vn30_df = pd.concat([vn30_df, vn30_dict[stock][columns]],  ignore_index=True)

In [147]:
vn30_df

,Symbol,TradingDate,Open,High,Low,Close,Volume,Value
0,ACB,02/01/2024,20118,20790,20118,20538,13883500,339648910000.0030
1,ACB,03/01/2024,20580,21000,20370,21000,9803400,241729125000.0010
2,ACB,04/01/2024,21168,21546,21084,21252,23587800,599573919999.9950
3,ACB,05/01/2024,21252,21378,21042,21378,9272100,234276055000
4,ACB,08/01/2024,21588,21630,21168,21294,12389600,314879939999.9990
...,...,...,...,...,...,...,...,...
7495,VNM,25/12/2024,63898,64394,63898,63898,1302500,84194640000.0002
7496,VNM,26/12/2024,64200,64200,63800,63800,1708800,109257980000
7497,VNM,27/12/2024,63900,64100,63700,63800,2443700,156296109999.9990
7498,VNM,30/12/2024,63600,63700,63300,63300,1490800,94622519999.9972


In [115]:
print(vn30_df.dtypes)

Symbol         object
TradingDate    object
Open           object
High           object
Low            object
Close          object
Volume         object
Value          object
dtype: object


In [117]:
vn30_df['TradingDate'] = pd.to_datetime(vn30_df['TradingDate'], format='%d/%m/%Y')

In [119]:
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Value']

for col in numeric_cols:
    vn30_df[col] = pd.to_numeric(vn30_df[col], errors='coerce')

In [121]:
vn30_df

,Symbol,TradingDate,Open,High,Low,Close,Volume,Value
0,ACB,2024-01-02,20118,20790,20118,20538,13883500,3.396489e+11
1,ACB,2024-01-03,20580,21000,20370,21000,9803400,2.417291e+11
2,ACB,2024-01-04,21168,21546,21084,21252,23587800,5.995739e+11
3,ACB,2024-01-05,21252,21378,21042,21378,9272100,2.342761e+11
4,ACB,2024-01-08,21588,21630,21168,21294,12389600,3.148799e+11
...,...,...,...,...,...,...,...,...
7495,VNM,2024-12-25,63898,64394,63898,63898,1302500,8.419464e+10
7496,VNM,2024-12-26,64200,64200,63800,63800,1708800,1.092580e+11
7497,VNM,2024-12-27,63900,64100,63700,63800,2443700,1.562961e+11
7498,VNM,2024-12-30,63600,63700,63300,63300,1490800,9.462252e+10


In [123]:
# Chuyển đổi cột TradingDate từ chuỗi (string) sang kiểu dữ liệu datetime của Pandas, giúp dễ dàng xử lý và phân tích thời gian.
vn30_df = vn30_df.pivot_table(index='TradingDate', columns='Symbol', values=['Open', 'High', 'Low', 'Close', 'Volume', 'Value'])

In [125]:
vn30_df

Close                                                         \
Symbol           ACB      BCM      BID      BVH      CTG       FPT      GAS   
TradingDate                                                                   
2024-01-02   20538.0  60972.0  35535.0  38693.0  27200.0   82248.0  68796.0   
2024-01-03   21000.0  60578.0  36196.0  38889.0  27600.0   82591.0  69160.0   
2024-01-04   21252.0  59888.0  36072.0  39182.0  28600.0   83363.0  69797.0   
2024-01-05   21378.0  59593.0  36775.0  39084.0  29000.0   83534.0  70252.0   
2024-01-08   21294.0  59396.0  38345.0  39084.0  29550.0   83448.0  69888.0   
...              ...      ...      ...      ...      ...       ...      ...   
2024-12-25   25450.0  69300.0  38650.0  53600.0  38200.0  150800.0  68400.0   
2024-12-26   25550.0  69700.0  39000.0  52600.0  38200.0  149800.0  68500.0   
2024-12-27   25550.0  69800.0  39150.0  52300.0  38800.0  149600.0  68500.0   
2024-12-30   25400.0  69800.0  38250.0  52000.0  38900.0  150500.0  68200.0   
2024-12-31   25800.0  70900.0  37550.0  50700.0  37800.0  152500.0  68100.0   

                                        ...      Volume              \
Symbol           GVR      HDB      HPG  ...         SSI         STB   
TradingDate                             ...                           
2024-01-02   20757.0  15928.0  24955.0  ...  18052300.0  14813300.0   
2024-01-03   21401.0  16008.0  25273.0  ...  11408700.0  19990700.0   
2024-01-04   21203.0  16328.0  25228.0  ...  58713300.0  31414000.0   
2024-01-05   21599.0  16408.0  25273.0  ...  25137200.0  31291500.0   
2024-01-08   21302.0  16448.0  25318.0  ...  20729000.0  15853600.0   
...              ...      ...      ...  ...         ...         ...   
2024-12-25   31100.0  23900.0  27150.0  ...  26221100.0  24376900.0   
2024-12-26   30900.0  24150.0  27000.0  ...   8764400.0  16442000.0   
2024-12-27   30700.0  24950.0  26850.0  ...  16346900.0  11145700.0   
2024-12-30   30550.0  26650.0  26750.0  ...  11494000.0  10631900.0   
2024-12-31   30350.0  25500.0  26650.0  ...   8234000.0   9388400.0   

                                                                       \
Symbol              TCB         TPB        VCB        VHM         VIB   
TradingDate                                                             
2024-01-02    3710100.0   6970700.0  1785800.0  7588500.0   4383500.0   
2024-01-03    5610600.0   7154500.0  1373000.0  6112600.0   4469800.0   
2024-01-04   10073900.0  30598000.0  2657900.0  8778000.0  10162800.0   
2024-01-05    3569900.0  11024500.0  1180300.0  4508700.0   4437400.0   
2024-01-08    6323800.0  10132200.0  1607800.0  6213900.0   6160100.0   
...                 ...         ...        ...        ...         ...   
2024-12-25   18370600.0  14525600.0  1827700.0  3559700.0  15320100.0   
2024-12-26    5475600.0   6862500.0  2202900.0  3690900.0  21867900.0   
2024-12-27   15395600.0  27796400.0  1639900.0  2847200.0  22848400.0   
2024-12-30   10145800.0   6732600.0  1706700.0  2516300.0   7629400.0   
2024-12-31    9403700.0   7930000.0  2374000.0  4191900.0   6423000.0   

                                             
Symbol             VIC       VJC        VNM  
TradingDate                                  
2024-01-02   2281300.0  929800.0  2142800.0  
2024-01-03   2275100.0  882300.0  1889800.0  
2024-01-04   2337800.0  667500.0  3401500.0  
2024-01-05   1481600.0  633100.0  1475700.0  
2024-01-08   2534400.0  626300.0  2396400.0  
...                ...       ...        ...  
2024-12-25   1908100.0  938200.0  1302500.0  
2024-12-26    877500.0  620800.0  1708800.0  
2024-12-27   1497700.0  650900.0  2443700.0  
2024-12-30   1415300.0  751900.0  1490800.0  
2024-12-31   1784400.0  684300.0  1641900.0  

[250 rows x 180 columns]

In [127]:
vn30_df.Close

Symbol,ACB,BCM,BID,BVH,CTG,FPT,GAS,GVR,HDB,HPG,...,SSI,STB,TCB,TPB,VCB,VHM,VIB,VIC,VJC,VNM
TradingDate,,,,,,,,,,,,,,,,,,,,,
2024-01-02,20538.0,60972.0,35535.0,38693.0,27200.0,82248.0,68796.0,20757.0,15928.0,24955.0,...,25373.0,27750.0,15565.0,14027.0,83500.0,43000.0,15975.0,44000.0,107000.0,64652.0
2024-01-03,21000.0,60578.0,36196.0,38889.0,27600.0,82591.0,69160.0,21401.0,16008.0,25273.0,...,25724.0,28500.0,15808.0,14148.0,84500.0,43400.0,16218.0,44150.0,107700.0,65315.0
2024-01-04,21252.0,59888.0,36072.0,39182.0,28600.0,83363.0,69797.0,21203.0,16328.0,25228.0,...,26232.0,28600.0,16050.0,14513.0,85900.0,43450.0,16460.0,44150.0,107800.0,65315.0
2024-01-05,21378.0,59593.0,36775.0,39084.0,29000.0,83534.0,70252.0,21599.0,16408.0,25273.0,...,26661.0,29300.0,16099.0,14594.0,86200.0,43100.0,16702.0,44100.0,107600.0,65125.0
2024-01-08,21294.0,59396.0,38345.0,39084.0,29550.0,83448.0,69888.0,21302.0,16448.0,25318.0,...,26622.0,29200.0,16535.0,14554.0,86800.0,43350.0,16863.0,44350.0,106600.0,64557.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-25,25450.0,69300.0,38650.0,53600.0,38200.0,150800.0,68400.0,31100.0,23900.0,27150.0,...,26550.0,36250.0,24450.0,16400.0,92400.0,40600.0,19200.0,40650.0,102000.0,63898.0
2024-12-26,25550.0,69700.0,39000.0,52600.0,38200.0,149800.0,68500.0,30900.0,24150.0,27000.0,...,26350.0,37100.0,24350.0,16550.0,92100.0,40650.0,19750.0,40650.0,101500.0,63800.0
2024-12-27,25550.0,69800.0,39150.0,52300.0,38800.0,149600.0,68500.0,30700.0,24950.0,26850.0,...,26600.0,37400.0,24600.0,16750.0,92200.0,40350.0,20050.0,40550.0,101200.0,63800.0


In [129]:

vn30_df.reset_index(inplace = True)

In [131]:
vn30_df

TradingDate    Close                                                \
Symbol                  ACB      BCM      BID      BVH      CTG       FPT   
0       2024-01-02  20538.0  60972.0  35535.0  38693.0  27200.0   82248.0   
1       2024-01-03  21000.0  60578.0  36196.0  38889.0  27600.0   82591.0   
2       2024-01-04  21252.0  59888.0  36072.0  39182.0  28600.0   83363.0   
3       2024-01-05  21378.0  59593.0  36775.0  39084.0  29000.0   83534.0   
4       2024-01-08  21294.0  59396.0  38345.0  39084.0  29550.0   83448.0   
..             ...      ...      ...      ...      ...      ...       ...   
245     2024-12-25  25450.0  69300.0  38650.0  53600.0  38200.0  150800.0   
246     2024-12-26  25550.0  69700.0  39000.0  52600.0  38200.0  149800.0   
247     2024-12-27  25550.0  69800.0  39150.0  52300.0  38800.0  149600.0   
248     2024-12-30  25400.0  69800.0  38250.0  52000.0  38900.0  150500.0   
249     2024-12-31  25800.0  70900.0  37550.0  50700.0  37800.0  152500.0   

                                   ...      Volume                          \
Symbol      GAS      GVR      HDB  ...         SSI         STB         TCB   
0       68796.0  20757.0  15928.0  ...  18052300.0  14813300.0   3710100.0   
1       69160.0  21401.0  16008.0  ...  11408700.0  19990700.0   5610600.0   
2       69797.0  21203.0  16328.0  ...  58713300.0  31414000.0  10073900.0   
3       70252.0  21599.0  16408.0  ...  25137200.0  31291500.0   3569900.0   
4       69888.0  21302.0  16448.0  ...  20729000.0  15853600.0   6323800.0   
..          ...      ...      ...  ...         ...         ...         ...   
245     68400.0  31100.0  23900.0  ...  26221100.0  24376900.0  18370600.0   
246     68500.0  30900.0  24150.0  ...   8764400.0  16442000.0   5475600.0   
247     68500.0  30700.0  24950.0  ...  16346900.0  11145700.0  15395600.0   
248     68200.0  30550.0  26650.0  ...  11494000.0  10631900.0  10145800.0   
249     68100.0  30350.0  25500.0  ...   8234000.0   9388400.0   9403700.0   

                                                                           \
Symbol         TPB        VCB        VHM         VIB        VIC       VJC   
0        6970700.0  1785800.0  7588500.0   4383500.0  2281300.0  929800.0   
1        7154500.0  1373000.0  6112600.0   4469800.0  2275100.0  882300.0   
2       30598000.0  2657900.0  8778000.0  10162800.0  2337800.0  667500.0   
3       11024500.0  1180300.0  4508700.0   4437400.0  1481600.0  633100.0   
4       10132200.0  1607800.0  6213900.0   6160100.0  2534400.0  626300.0   
..             ...        ...        ...         ...        ...       ...   
245     14525600.0  1827700.0  3559700.0  15320100.0  1908100.0  938200.0   
246      6862500.0  2202900.0  3690900.0  21867900.0   877500.0  620800.0   
247     27796400.0  1639900.0  2847200.0  22848400.0  1497700.0  650900.0   
248      6732600.0  1706700.0  2516300.0   7629400.0  1415300.0  751900.0   
249      7930000.0  2374000.0  4191900.0   6423000.0  1784400.0  684300.0   

                   
Symbol        VNM  
0       2142800.0  
1       1889800.0  
2       3401500.0  
3       1475700.0  
4       2396400.0  
..            ...  
245     1302500.0  
246     1708800.0  
247     2443700.0  
248     1490800.0  
249     1641900.0  

[250 rows x 181 columns]

In [137]:
# Xuất vn30_df ra file csv. Vị trí lưu thay đổi tùy chọn
file_path = r'C:\Users\DELL\Desktop\vn30_2024.csv'
vn30_df.to_csv(file_path, index=False)